In [52]:
import numpy as np
from urllib.parse import urlparse
import pandas as pd
from pyspark.sql import SparkSession
from tqdm import tqdm 

spark = SparkSession.builder.appName("test").getOrCreate()

def netloc(link):
    return urlparse(link)[1]

df = pd.read_json("News_Category_Dataset_v3.json", lines=True)
df['source'] = df['link'].map(netloc)


In [50]:
df_sp = spark.createDataFrame(df)
df = df_sp.limit(1000).toPandas()

In [68]:
from nela_features.nela_features import NELAFeatureExtractor

nela = NELAFeatureExtractor()

# Extract all feature groups at once
#feature_vector, feature_names = nela.extract_all(newsarticle)
pbar = tqdm(total=1000)
def nela_with_bar(text):
    pbar.update(1)
    return nela.extract_all(text)
#df['nela'] = df['short_description'].map(nela_with_bar)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [65]:
#df.to_csv("1000rowswithnela.csv")

In [66]:
data = df.to_numpy()

vect_nela = np.vectorize(nela_with_bar)
nela_features = vect_nela(data[:,3])

1731it [32:42,  1.11s/it]                                                                                              

KeyboardInterrupt: 

In [42]:
print(len(data))
print(df.source.unique())
print(df.columns)

209527
['www.huffpost.com' 'www.huffingtonpost.in' 'www.huffingtonpost.com'
 'www.huffingtonpost.comhttps:' 'www.huffingtonpost.comhttp:']
Index(['link', 'headline', 'category', 'short_description', 'authors', 'date',
       'source'],
      dtype='object')


In [43]:
features:
for row in data:
    

['https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9'
 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters'
 'U.S. NEWS'
 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'
 'Carla K. Johnson, AP' Timestamp('2022-09-23 00:00:00')
 'www.huffpost.com']


In [45]:
from sklearn.model_selection import train_test_split

# splitting into two matrices of second matrix by size
second_size = 100000/len(data)

X1, X2 = train_test_split(data, test_size=second_size)

print('X1 shape: ', X1.shape)
print('X2 shape: ', X2.shape)

X1 shape:  (109527, 7)
X2 shape:  (100000, 7)
